In [2]:
import pdf2image
from pdf2image import convert_from_path
import cv2
import numpy as np

In [ ]:
images=convert_from_path("D:\sagar\Sanskrit_Text.pdf",500,poppler_path=r'C:\Program Files\poppler-23.01.0\Library\bin')
for i in range(len(images)):
    images[i].save('pages'+str(i)+'.jpg','JPEG')

In [22]:
import json
# Load the image
img = cv2.imread("D:\sagar\pages0.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to convert the image to a binary format
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Apply morphological transformations to remove noise and connect the text
kernel = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

# Find the contours of the text
contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Define a function to get the bounding box coordinates for each contour
def get_bounding_box(contour):
    x, y, w, h = cv2.boundingRect(contour)
    return x, y, w, h

# Sort the contours from top to bottom
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[1])

# Define a list to store the bounding box coordinates for each line
line_coords_list = []

# Loop through each contour and group them by lines
for contour in contours:
    x1, y1, w, h = get_bounding_box(contour)
    x2 = x1 + w
    y2 = y1 + h
    # Check if the current contour is on the same line as the previous one
    if line_coords_list and y1 <= line_coords_list[-1][3]:
        # Update the bounding box coordinates for the current line
        line_coords_list[-1] = (min(line_coords_list[-1][0], x1), min(line_coords_list[-1][1], y1), max(line_coords_list[-1][2], x2), max(line_coords_list[-1][3], y2))
    else:
        # Create a new bounding box for the current line
        line_coords_list.append((x1, y1, x2, y2))

# Create a dictionary to store the line information
line_dict = {}

# Create a new image for each line with its bounding box
for i, line_coords in enumerate(line_coords_list):
    x1_line, y1_line, x2_line, y2_line = line_coords
    # Draw a rectangle around the current line
    cv2.rectangle(img, (x1_line, y1_line), (x2_line, y2_line), (0, 255, 0), 2)
    # Extract the current line image
    line_img = img[y1_line:y2_line, x1_line:x2_line]
    # Save the line image
    cv2.imwrite(f"line_{i+1}.jpg", line_img)
    # Add the bounding box coordinates to the line dictionary
    line_dict[f"line_{i+1}"] = {
        "x1": x1_line,
        "y1": y1_line,
        "x2": x2_line,
        "y2": y2_line
    }

# Save the line information as a JSON file
with open("line_info.json", "w") as f:
    json.dump(line_dict, f)

# Show the original image with the bounding boxes
cv2.imshow("D:\sagar\line_info.json", img)
cv2.waitKey(0)

-1